In [1]:
import os
import torch
# 打印当前工作目录
print("当前工作目录:", os.getcwd())
# 更改工作目录到指定路径
os.chdir('/root/autodl-tmp/RPG-DiffusionMaster-main')
# 再次打印当前工作目录以确认更改
print("更改后的工作目录:", os.getcwd())

当前工作目录: /root/autodl-tmp/RPG-DiffusionMaster-main
更改后的工作目录: /root/autodl-tmp/RPG-DiffusionMaster-main


In [2]:
from RegionalDiffusion_base import RegionalDiffusionPipeline
from RegionalDiffusion_xl import RegionalDiffusionXLPipeline
from diffusers.schedulers import KarrasDiffusionSchedulers,DPMSolverMultistepScheduler
from transformers import MllamaForConditionalGeneration, PreTrainedTokenizerFast
from mllm import local_llm,GPT4
import random
import torch

In [3]:
pipe = RegionalDiffusionXLPipeline.from_single_file("../mod/albedobaseXL_v20.safetensors", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")

Some weights of the model checkpoint were not used when initializing CLIPTextModel: 
 ['text_model.embeddings.position_ids']
Some weights of the model checkpoint were not used when initializing CLIPTextModelWithProjection: 
 ['text_model.embeddings.position_ids']



开始init了！！！


In [4]:
pipe.to("cuda")

RegionalDiffusionXLPipeline {
  "_class_name": "RegionalDiffusionXLPipeline",
  "_diffusers_version": "0.27.2",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [5]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config,use_karras_sigmas=True)

In [6]:
pipe.enable_xformers_memory_efficient_attention()

In [64]:
prompt= "A vibrant, playful, and cozy space."

In [65]:
para_dict = GPT4(prompt,key='sk-UDZJFwAlNjn5bnQOOEJn8W7JXvm225kzvOXZARvVtuiQnPnR')

yuanming test 1119 waiting for GPT-4 response


In [66]:
split_ratio = para_dict['Final split ratio'] 

In [67]:
print(split_ratio)

0.5,0.5,0.5;0.5,0.5,0.5


In [68]:
regional_prompt = para_dict['Regional Prompt'] 

In [69]:
print(regional_prompt)

a star or animal-shaped night light glows softly BREAK an interactive wall with a chalkboard or magnets BREAK a colorful foam puzzle mat BREAK a cozy children’s bed with safety rails


In [70]:
negative_prompt = "NSFW,worst quality,low quality,normal quality,lowres,monochrome,grayscale,skin spots,acnes,skin blemishes,age spot,ugly,duplicate,morbid,mutilated,tranny,mutated hands,poorly drawn hands,blurry,bad anatomy,bad proportions,extra limbs,disfigured,missing arms,extra legs,fused fingers,too many fingers,unclear eyes,lowers,bad hands,missing fingers,extra digit,bad hands,missing fingers,extra arms and legs"

In [71]:
random_seed = random.randint(0, 100000)
images = pipe(
    prompt=regional_prompt,
    split_ratio=split_ratio, # The ratio of the regional prompt, the number of prompts is the same as the number of regions
    batch_size = 2, #batch size
    base_ratio = 0.3, # The ratio of the base prompt
    base_prompt= prompt,
    #base_prompt=None,
    num_inference_steps=20, # sampling step
    #height = 1024,
    height = 1600,    #调个特殊值试试
    negative_prompt=negative_prompt, # negative prompt
    #width = 1024,
    width = 1600,    #调个特殊值试试
    seed = random_seed,# random seed
    guidance_scale = 7.0
).images[0]


开始call方法了!!!

开始keyconverter!!!

开始matrixdealer!!!
split_ratio 0.5,0.5,0.5;0.5,0.5,0.5
*******************************************************************************************************

r 0
c 0
[[[0, 0.5], [0.5, 1.0]], [[0, 0.5], [0.5, 1.0]]]
start:: 0 end:: 0.5 base:: 0.3 breaks:: 0
r 0
c 1
[[[0, 0.5], [0.5, 1.0]], [[0, 0.5], [0.5, 1.0]]]
start:: 0.5 end:: 1.0 base:: 0.3 breaks:: 0
ooo 0 0.5
row:: start:: 0 end 0.5


r 1
c 0
[[[0, 0.5], [0.5, 1.0]], [[0, 0.5], [0.5, 1.0]]]
start:: 0 end:: 0.5 base:: 0.3 breaks:: 0
r 1
c 1
[[[0, 0.5], [0.5, 1.0]], [[0, 0.5], [0.5, 1.0]]]
start:: 0.5 end:: 1.0 base:: 0.3 breaks:: 0
ooo 0.5 1.0
row:: start:: 0.5 end 1.0


********************************************************************************************************
self.split_ratio [<matrix.Row object at 0x7ff5831be370>, <matrix.Row object at 0x7ff582f160d0>]
self.base_ratio 0.3
**************开始后面的部分了****************************************************************************************

  0%|          | 0/20 [00:00<?, ?it/s]

循环完了
output_type pil
******************************************************************************************************************************************************************
Processing Image 1
Element 0: start = 0, end = 0.5
  Col 0: start = 0, end = 0.5, base = 0.3
  Col 1: start = 0.5, end = 1.0, base = 0.3
Element 1: start = 0.5, end = 1.0
  Col 0: start = 0, end = 0.5, base = 0.3
  Col 1: start = 0.5, end = 1.0, base = 0.3
img <class 'PIL.Image.Image'>
******************************************************************************************************************************************************************


In [72]:
images.save("testlunwen.png")